## LINKEDIN ETL / WEBSCRAPING

In [73]:
import pandas as pd
from bs4 import BeautifulSoup
import random
import requests

In [74]:
titulo = 'data'
loc = 'br'
start = 0
f_TPR = "r604800" # Filtro de tempo; r604800 = no máximo uma semana atrás

In [75]:
lista_de_vagas = []

# Coleta no máximo 1000 vagas (filtrando no período de uma semana)
while start < 1000:
    
    # URL encontrada no modo desenvolvedor do navegador; start=0 começa a lista na primeira vaga; f_TPR filtra vagas anunciadas à menos de uma semana
    list_url = f"https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords={titulo}&location={loc}&f_TPR={f_TPR}&start={start}"

    response = requests.get(list_url)
    
    # Ler HTML
    list_soup = BeautifulSoup(response.text, "html.parser")

    #Lista de 'li'
    lista_de_vagas.extend(list_soup.find_all('li'))
    start += 10
else:
    print(f"Todas as vagas foram processadas! Número de vagas:{len(lista_de_vagas)}")
    start = 0

Todas as vagas foram processadas! Número de vagas:1000


## Encontrar ID de cada vaga

In [76]:
''' 
<div class="base-card relative w-full hover:no-underline focus:no-underline base-card--link base-search-card base-search-card--link job-search-card"
data-entity-urn="urn:li:jobPosting:6666666666" data-impression-id="jobs-search-result-0"
data-reference-id="33XOgxFbhjo/19k2K/85tw==" data-tracking-id="hGv1pZGEVUHn+yeZTU+/5g==" data-column="1" data-row="1">

data-entity-urn="urn:li:jobPosting:6666666666

'''

' \n<div class="base-card relative w-full hover:no-underline focus:no-underline base-card--link base-search-card base-search-card--link job-search-card"\ndata-entity-urn="urn:li:jobPosting:6666666666" data-impression-id="jobs-search-result-0"\ndata-reference-id="33XOgxFbhjo/19k2K/85tw==" data-tracking-id="hGv1pZGEVUHn+yeZTU+/5g==" data-column="1" data-row="1">\n\ndata-entity-urn="urn:li:jobPosting:6666666666\n\n'

##### Criar lista com ID de cada vaga

In [77]:
lista_de_ids = []

for vaga in lista_de_vagas:
    
    # Div contendo ID da vaga
    base_card_full_link = vaga.find("div", {"class": "base-card"})
    
    # If para não gerar erro para vagas sem ID
    if base_card_full_link and base_card_full_link.get("data-entity-urn"):
        id_vaga = base_card_full_link.get("data-entity-urn").split(":")[3]
        lista_de_ids.append(id_vaga)
    else:
        continue
    
len(lista_de_ids)

994

##### Com o ID, acessar o link de cada vaga e extrair informações

In [78]:
# Quero extrair o título, há quanto tempo foi postada, e o link da vaga para um DF
# Depois é possível utilizar as descrições para filtrar vagas relevantes para mim
# É possível também utilizar as descrições para fazer uma análise dos requisitos e ferramentas mais pedidas

lista_vagas = []

for id in lista_de_ids:
    url_vaga = f"https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/{id}"
    vaga_response = requests.get(url_vaga)
    vaga_soup = BeautifulSoup(vaga_response.text, "html.parser")
    
    # Dicionário para guardar todas as informações da vaga
    vaga_info = {}
    
    # Empresa
    try:
        vaga_info['empresa'] = vaga_soup.find("img", {"class": "artdeco-entity-image artdeco-entity-image--square-5"}).get("alt").strip()
    except:
        vaga_info['empresa'] = None
    # Título da vaga
    try:
        vaga_info['vaga'] = vaga_soup.find("h2", {"class": "top-card-layout__title font-sans text-lg papabear:text-xl font-bold leading-open text-color-text mb-0 topcard__title"}).text.strip()
    except:
        vaga_info['vaga'] = None
    # Tempo postada
    try:
        vaga_info["data"] = vaga_soup.find("span", {"class": "posted-time-ago__text topcard__flavor--metadata"}).text.strip()
    except:
        vaga_info["data"] = None
    # Link
    vaga_info['link'] = url_vaga
    # Descrição da vaga
    try:
        descricao = vaga_soup.find("div", {"class": "show-more-less-html__markup"}).get_text(separator=" ", strip=True)
    except:
        descricao = None  # Corrigido para manter a variável

    # Criterios da vaga
    try:
        criterios = vaga_soup.find("div", {"class": "description__job-criteria-list"}).get_text(separator=" ", strip=True)
    except:
        criterios = None

    # Combinar descrição e critérios
    if descricao and criterios:
        vaga_info['descricao'] = f"{descricao} {criterios}"
    elif descricao:
        vaga_info['descricao'] = descricao
    elif criterios:
        vaga_info['descricao'] = criterios
    else:
        vaga_info['descricao'] = None
        
    lista_vagas.append(vaga_info)
    
lista_vagas = pd.DataFrame(lista_vagas)

lista_vagas     



,empresa,vaga,data,link,descricao
0,Itaú Unibanco,Faça sua Carreira de Dados no Itaú 🚀🧡,5 days ago,https://www.linkedin.com/jobs-guest/jobs/api/j...,Vem construir a área de Dados no Itaú_ Nossa m...
1,Senai São Paulo,SESI - AUXILIAR DE ESPORTE DE RENDIMENTO - BAS...,None,https://www.linkedin.com/jobs-guest/jobs/api/j...,Processo seletivo para a cidade de Araraquara-...
2,Globo,Analista de Dados Júnior | Performance,3 days ago,https://www.linkedin.com/jobs-guest/jobs/api/j...,Job Description A Globo é feita de gente que q...
3,Mundiale,DATA ANALYST,5 days ago,https://www.linkedin.com/jobs-guest/jobs/api/j...,✨ Nossa missão aqui na Mundiale é simplificar ...
4,GEES S/A,Assistente de Estoque Gees Defensivos Balsas - MA,None,https://www.linkedin.com/jobs-guest/jobs/api/j...,Controlar o recebimento de produtos e materiai...
...,...,...,...,...,...
989,Foco do Cliente,Vaga Freelancer - AUDITORIA - PR,None,https://www.linkedin.com/jobs-guest/jobs/api/j...,Estamos recrutando auditores para participar d...
990,Outlier,AI Math Specialist,3 days ago,https://www.linkedin.com/jobs-guest/jobs/api/j...,Outlier helps the world’s most innovative comp...
991,Outlier,Math Educator - Remote,3 days ago,https://www.linkedin.com/jobs-guest/jobs/api/j...,Outlier helps the world’s most innovative comp...
992,Outlier,Math Educator - Remote,3 days ago,https://www.linkedin.com/jobs-guest/jobs/api/j...,Outlier helps the world’s most innovative comp...


## Tratamento de dados

In [79]:
import re
import unicodedata
from datetime import datetime, timedelta

In [ ]:
# Palavras-chave
palavras_chave = ['dado', 'dados', 'data', 'analis', 'analys']
regex = '|'.join(palavras_chave)
# Filtra o DataFrame
df = lista_vagas[lista_vagas['vaga'].str.contains(regex, case=False, na=False)]
# Reseta o índice após o filtro
df.reset_index(drop=True, inplace=True)

# Função para limpar texto e deixar em minúsculo
def limpar_texto(texto):
    if isinstance(texto, str):
        # Remove tudo que não seja letra, número ou espaço e converte para minúsculo
        texto = unicodedata.normalize('NFKD', texto).encode('ASCII', 'ignore').decode('utf-8')
        return re.sub(r'[^a-zA-Z0-9\s]', '', texto).lower()
    return texto

# Aplicando ao DataFrame
df = df.apply(lambda col: col if col.name == 'link' else col.apply(limpar_texto))


##### Formatação da coluna de data

In [81]:
hoje = datetime.today()

def converter_data(texto):
    if pd.isnull(texto):
        return hoje.strftime('%d/%m/%Y')  # Retorna a data atual se for nulo

    match = re.search(r'(\d+)\sday[s]?', texto)
    if match:
        dias = int(match.group(1))
        nova_data = hoje - timedelta(days=dias)
        return nova_data.strftime('%d/%m/%Y')
    return hoje.strftime('%d/%m/%Y')  # Retorna a data atual se não houver correspondência

# Aplica a função
df['data'] = df['data'].apply(converter_data)


In [82]:
df

,empresa,vaga,data,link,descricao
0,itau unibanco,faca sua carreira de dados no itau,17/02/2025,https://www.linkedin.com/jobs-guest/jobs/api/j...,vem construir a area de dados no itau nossa mi...
1,globo,analista de dados junior performance,19/02/2025,https://www.linkedin.com/jobs-guest/jobs/api/j...,job description a globo e feita de gente que q...
2,mundiale,data analyst,17/02/2025,https://www.linkedin.com/jobs-guest/jobs/api/j...,nossa missao aqui na mundiale e simplificar a...
3,libbs farmaceutica ltda,analista melhoria da jornada do colaborador ju...,17/02/2025,https://www.linkedin.com/jobs-guest/jobs/api/j...,somos uma empresa farmaceutica brasileira que ...
4,unimed fortaleza,pessoa analista de dados junior,21/02/2025,https://www.linkedin.com/jobs-guest/jobs/api/j...,job description ei ja pensou em fazer parte de...
...,...,...,...,...,...
227,picpay,analista de dados pl shop vaga afirmativa pcd,22/02/2025,https://www.linkedin.com/jobs-guest/jobs/api/j...,sobre o picpay com mais de dez anos de histori...
228,agileengine,python data engineer junior id31594,21/02/2025,https://www.linkedin.com/jobs-guest/jobs/api/j...,agileengine is one of the inc 5000 fastestgrow...
229,dexian,big data specialist,17/02/2025,https://www.linkedin.com/jobs-guest/jobs/api/j...,quem somos a dexian lancada em 2023 tem presen...
230,bencorp,analista sr health analytics,18/02/2025,https://www.linkedin.com/jobs-guest/jobs/api/j...,voce que e disruptivo tem empatia cordialidade...


In [83]:
senioridades = ['estagio', 'estagiario', 'assistente', 'junior', 'pleno', 'senior', 'jr', 'sr', 'pl', 'pleno-senior']

def identificar_senioridade(texto):
    texto_lower = texto.lower()
    for nivel in senioridades:
        # Usa regex para capturar variações como "jr", "jr.", etc.
        if re.search(rf'\b{nivel}\b', texto_lower):
            return nivel.capitalize()
    return 'Não especificado'

In [84]:
df['senioridade'] = df.apply(
    lambda x: identificar_senioridade(str(x['descricao']) + ' ' + str(x['vaga'])), axis=1
)

In [85]:
df

,empresa,vaga,data,link,descricao,senioridade
0,itau unibanco,faca sua carreira de dados no itau,17/02/2025,https://www.linkedin.com/jobs-guest/jobs/api/j...,vem construir a area de dados no itau nossa mi...,Não especificado
1,globo,analista de dados junior performance,19/02/2025,https://www.linkedin.com/jobs-guest/jobs/api/j...,job description a globo e feita de gente que q...,Junior
2,mundiale,data analyst,17/02/2025,https://www.linkedin.com/jobs-guest/jobs/api/j...,nossa missao aqui na mundiale e simplificar a...,Não especificado
3,libbs farmaceutica ltda,analista melhoria da jornada do colaborador ju...,17/02/2025,https://www.linkedin.com/jobs-guest/jobs/api/j...,somos uma empresa farmaceutica brasileira que ...,Junior
4,unimed fortaleza,pessoa analista de dados junior,21/02/2025,https://www.linkedin.com/jobs-guest/jobs/api/j...,job description ei ja pensou em fazer parte de...,Junior
...,...,...,...,...,...,...
227,picpay,analista de dados pl shop vaga afirmativa pcd,22/02/2025,https://www.linkedin.com/jobs-guest/jobs/api/j...,sobre o picpay com mais de dez anos de histori...,Pl
228,agileengine,python data engineer junior id31594,21/02/2025,https://www.linkedin.com/jobs-guest/jobs/api/j...,agileengine is one of the inc 5000 fastestgrow...,Junior
229,dexian,big data specialist,17/02/2025,https://www.linkedin.com/jobs-guest/jobs/api/j...,quem somos a dexian lancada em 2023 tem presen...,Não especificado
230,bencorp,analista sr health analytics,18/02/2025,https://www.linkedin.com/jobs-guest/jobs/api/j...,voce que e disruptivo tem empatia cordialidade...,Sr


In [86]:
# 🔍 Lista de ferramentas e skills
skills_tools = {
    # 📊 Linguagens de Programação
    'python', 'r', 'sql', 'scala', 'java', 'c++', 'c#', 'javascript', 'typescript', 'go', 'bash',

    # 🏗️ Frameworks & Bibliotecas
    'pandas', 'numpy', 'matplotlib', 'seaborn', 'scikit-learn', 'statsmodels', 'tensorflow', 'keras', 
    'pytorch', 'xgboost', 'lightgbm', 'catboost', 'nltk', 'spacy', 'openai', 'transformers',

    # 📊 BI & Visualização
    'power bi', 'tableau', 'looker', 'qlikview', 'metabase', 'superset',

    # ☁️ Cloud & Big Data
    'aws', 'azure', 'google cloud', 'databricks', 'redshift', 'bigquery', 'snowflake', 'hadoop', 'spark', 
    'hive', 'data lake', 's3', 'lambda', 'emr',

    # 🔄 ETL, Orquestração & DataOps
    'airflow', 'luigi', 'talend', 'informatica', 'dbt', 'nifi', 'kafka', 'flink', 'kinesis', 'glue',

    # 🛢️ Bancos de Dados
    'mysql', 'postgresql', 'sql server', 'oracle', 'mongodb', 'redis', 'cassandra', 'neo4j', 'dynamodb',

    # 🐳 DevOps & Infraestrutura
    'docker', 'kubernetes', 'terraform', 'ansible', 'jenkins', 'ci/cd', 'git', 'github', 'gitlab', 'bitbucket',

    # 🤖 Inteligência Artificial & Ciência de Dados
    'machine learning', 'deep learning', 'nlp', 'computer vision', 'estatística', 'probabilidade', 
    'regressão', 'classificação', 'clusterização', 'modelagem preditiva', 'análise exploratória', 
    'engenharia de atributos', 'visualização de dados', 'otimização', 'séries temporais', 'reinforcement learning',

    # 🔐 Data Governance & Segurança
    'data governance', 'lgpd', 'gdpr', 'segurança da informação', 'mascaramento de dados', 'data lineage',

    # 📈 Metodologias & Conceitos
    'scrum', 'kanban', 'agile', 'lean', 'six sigma', 'okrs', 'kpi', 'etl', 'elt', 'data warehouse', 
    'data lakehouse', 'data mesh', 'arquitetura de dados', 'agil', 'ageis'

    # 📚 Soft Skills
    'trabalho em equipe', 'comunicação', 'resolução de problemas', 'pensamento crítico', 'criatividade',
    'liderança', 'gestão de tempo', 'empatia', 'proatividade', 'adaptabilidade', 'tomada de decisão', 
    'negociação', 'gestão de projetos', 'aprendizado contínuo', 'análise estratégica', 'foco em resultados', 
    'colaboração', 'inteligência emocional', 'pensamento analítico', 'orientação a detalhes'
}

In [87]:
def filtrar_skills(texto):
    texto_lower = texto.lower()  # Deixa tudo em minúsculo para padronizar
    return [skill for skill in skills_tools if skill in texto_lower]

In [88]:
df.descricao = df.descricao.apply(filtrar_skills)

In [89]:
df

,empresa,vaga,data,link,descricao,senioridade
0,itau unibanco,faca sua carreira de dados no itau,17/02/2025,https://www.linkedin.com/jobs-guest/jobs/api/j...,"[airflow, spark, aws, git, etl, machine learni...",Não especificado
1,globo,analista de dados junior performance,19/02/2025,https://www.linkedin.com/jobs-guest/jobs/api/j...,"[sql, criatividade, git, etl, kpi, java, pytho...",Junior
2,mundiale,data analyst,17/02/2025,https://www.linkedin.com/jobs-guest/jobs/api/j...,"[power bi, sql, agil, r]",Não especificado
3,libbs farmaceutica ltda,analista melhoria da jornada do colaborador ju...,17/02/2025,https://www.linkedin.com/jobs-guest/jobs/api/j...,"[power bi, databricks, sql, python, r]",Junior
4,unimed fortaleza,pessoa analista de dados junior,21/02/2025,https://www.linkedin.com/jobs-guest/jobs/api/j...,"[lgpd, data lake, go, r, proatividade]",Junior
...,...,...,...,...,...,...
227,picpay,analista de dados pl shop vaga afirmativa pcd,22/02/2025,https://www.linkedin.com/jobs-guest/jobs/api/j...,"[databricks, tableau, sql, kpi, python, go, r]",Pl
228,agileengine,python data engineer junior id31594,21/02/2025,https://www.linkedin.com/jobs-guest/jobs/api/j...,"[agile, airflow, azure, spark, sql, agil, etl,...",Junior
229,dexian,big data specialist,17/02/2025,https://www.linkedin.com/jobs-guest/jobs/api/j...,"[airflow, azure, spark, aws, sql, agil, git, c...",Não especificado
230,bencorp,analista sr health analytics,18/02/2025,https://www.linkedin.com/jobs-guest/jobs/api/j...,"[power bi, spark, lgpd, tableau, sql, modelage...",Sr


##### Exportar DataFrame como CSV

In [90]:
df.to_csv('vagas.csv')